In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim  

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS

from textblob import TextBlob

from tqdm import tqdm_notebook as tqdm
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

C:\Users\HP\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\Users\HP\Anaconda3\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict


In [2]:
df1 = pd.read_csv('ethiopia_tripadvisor_data_model_3.csv')
df1.head()

,Destination Name,Destination Type,Review Head,Review Body,Review Text,RB_Sentence1,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text,TextBlob Polarity RBS1,RB_Sentence1_Split
0,Rock-Hewn Churches of Lalibela,Landmarks,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,Amazing location must visit.,-0.05,0.300000,0.125000,0.6000,"['Amazing', 'location', 'must', 'visit.']"
1,Rock-Hewn Churches of Lalibela,Landmarks,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,I do recommend to visit Lalibela churches and ...,0.25,-0.150000,-0.016667,0.0000,"['I', 'do', 'recommend', 'to', 'visit', 'Lalib..."
2,Rock-Hewn Churches of Lalibela,Landmarks,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,A must visit in Ethiopia - one of the most uni...,0.90,0.345000,0.483750,0.4375,"['A', 'must', 'visit', 'in', 'Ethiopia', '-', ..."
3,Rock-Hewn Churches of Lalibela,Landmarks,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,These churches have to be seen to be believed.,0.60,0.225000,0.350000,0.0000,"['These', 'churches', 'have', 'to', 'be', 'see..."
4,Rock-Hewn Churches of Lalibela,Landmarks,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,Thanks to our tour organiser Ephram who is bas...,0.20,0.233333,0.228571,0.2000,"['Thanks', 'to', 'our', 'tour', 'organiser', '..."


In [3]:
df1['RH_Split'] = df1['Review Head'].apply(lambda x: x.split(' '))
df1['RB_Split'] = df1['Review Body'].apply(lambda x: x.split(' '))
df1['RT_Split'] = df1['Review Text'].apply(lambda x: x.split(' '))
df1['RB_Sentence1_Split'] = df1['RB_Sentence1'].apply(lambda x: x.split(' '))
df1.head()

,Destination Name,Destination Type,Review Head,Review Body,Review Text,RB_Sentence1,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text,TextBlob Polarity RBS1,RB_Sentence1_Split,RH_Split,RB_Split,RT_Split
0,Rock-Hewn Churches of Lalibela,Landmarks,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,Amazing location must visit.,-0.05,0.300000,0.125000,0.6000,"[Amazing, location, must, visit.]","[Incredible, locations, don't, be, put, off, t...","[Amazing, location, must, visit., This, site, ...","[Incredible, locations, don't, be, put, off, t..."
1,Rock-Hewn Churches of Lalibela,Landmarks,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,I do recommend to visit Lalibela churches and ...,0.25,-0.150000,-0.016667,0.0000,"[I, do, recommend, to, visit, Lalibela, church...","[It, is, a, real, miracle, of, the, world!]","[I, do, recommend, to, visit, Lalibela, church...","[It, is, a, real, miracle, of, the, world!, I,..."
2,Rock-Hewn Churches of Lalibela,Landmarks,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,A must visit in Ethiopia - one of the most uni...,0.90,0.345000,0.483750,0.4375,"[A, must, visit, in, Ethiopia, -, one, of, the...","[Incredible, experience, ]","[A, must, visit, in, Ethiopia, -, one, of, the...","[Incredible, experience, A, must, visit, in, E..."
3,Rock-Hewn Churches of Lalibela,Landmarks,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,These churches have to be seen to be believed.,0.60,0.225000,0.350000,0.0000,"[These, churches, have, to, be, seen, to, be, ...",[Amazing],"[These, churches, have, to, be, seen, to, be, ...","[Amazing, These, churches, have, to, be, seen,..."
4,Rock-Hewn Churches of Lalibela,Landmarks,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,Thanks to our tour organiser Ephram who is bas...,0.20,0.233333,0.228571,0.2000,"[Thanks, to, our, tour, organiser, Ephram, who...","[So, much, history]","[Thanks, to, our, tour, organiser, Ephram, who...","[So, much, history, Thanks, to, our, tour, org..."


In [4]:
df1['Destination Type'].value_counts()

Landmarks       3803
Museums         2507
Nature/Parks    2001
Name: Destination Type, dtype: int64

### Topic Modelling

In [5]:
# NLTK Stop words
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

test_subset = df1['RB_Split'].values
sid = SentimentIntensityAnalyzer()
pos_word_list=[]
neu_word_list=[]
neg_word_list=[]

for value in test_subset:
    for word in value:
        if (sid.polarity_scores(word)['compound']) >= 0.4:
            pos_word_list.append(word)
        elif (sid.polarity_scores(word)['compound']) <= -0.4:
            print
            neg_word_list.append(word)
        else:
            neu_word_list.append(word)                

print('Positive :', len(pos_word_list))       
print('Neutral :', len(neu_word_list))    
print('Negative :', len(neg_word_list))

Positive : 13151
Neutral : 309837
Negative : 1632


In [6]:
from nltk.corpus import stopwords

main_stop_words = ['ethiopia', 'ethiopian', 'rock', 'hewn', 'church', 'churches', 'park', 'parks', 'museum', 
                       'museums', 'omo', 'valley', 'mountain', 'mountains', 'addis', 'ababa', 'lucy', 'simien', 
                       'national', 'lalibela', 'gelada', 'haile', 'selassie', 'gondar', 'africa' , 'harar', 
                       'cathedral', 'cathedrals', 'place', 'visit', 'girmay', 
                       '\n', '‘s']
positive_stop_words = list(set(pos_word_list))
negative_stop_words = list(set(neg_word_list))
total_stop_words = main_stop_words + positive_stop_words + negative_stop_words

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words.extend(total_stop_words)
print('Stop Words: ', len(stop_words))

Stop Words:  1724


In [7]:
# Convert to list
data = df1['Review Body'].values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Amazing location must visit. This site is marketed as churches but I think '
 'it is actually a rock city as there are no records of when they were built '
 'and who built them. Must see.']


In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['amazing', 'location', 'must', 'visit', 'this', 'site', 'is', 'marketed', 'as', 'churches', 'but', 'think', 'it', 'is', 'actually', 'rock', 'city', 'as', 'there', 'are', 'no', 'records', 'of', 'when', 'they', 'were', 'built', 'and', 'who', 'built', 'them', 'must', 'see']]


In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['amazing', 'location', 'must', 'visit', 'this', 'site', 'is', 'marketed', 'as', 'churches', 'but', 'think', 'it', 'is', 'actually', 'rock', 'city', 'as', 'there', 'are', 'no', 'records', 'of', 'when', 'they', 'were', 'built', 'and', 'who', 'built', 'them', 'must', 'see']


In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['location', 'site', 'market', 'think', 'actually', 'city', 'record', 'build', 'build', 'see']]


In [12]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]


In [13]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('actually', 1),
  ('build', 2),
  ('city', 1),
  ('location', 1),
  ('market', 1),
  ('record', 1),
  ('see', 1),
  ('site', 1),
  ('think', 1)]]

In [14]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [15]:
# Print the Keyword in the n topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.083*"well" + 0.057*"lot" + 0.049*"exhibit" + 0.032*"organize" + '
  '0.024*"city" + 0.023*"learn" + 0.019*"exhibition" + 0.019*"campsite" + '
  '0.018*"building" + 0.017*"stay"'),
 (1,
  '0.098*"tour" + 0.046*"find" + 0.042*"view" + 0.035*"old" + 0.029*"travel" + '
  '0.029*"first" + 0.028*"culture" + 0.028*"country" + 0.023*"area" + '
  '0.023*"recommend"'),
 (2,
  '0.091*"trek" + 0.072*"baboon" + 0.052*"arrange" + 0.040*"location" + '
  '0.030*"detail" + 0.030*"run" + 0.028*"photo" + 0.028*"ever" + 0.026*"early" '
  '+ 0.025*"camping"'),
 (3,
  '0.051*"say" + 0.035*"long" + 0.030*"absolutely" + 0.028*"highlight" + '
  '0.028*"month" + 0.027*"historical" + 0.027*"informative" + 0.025*"full" + '
  '0.025*"guy" + 0.020*"ask"'),
 (4,
  '0.103*"night" + 0.067*"floor" + 0.052*"incredible" + 0.035*"man" + '
  '0.034*"would" + 0.024*"stop" + 0.023*"camp" + 0.022*"explore" + '
  '0.022*"present" + 0.018*"tradition"'),
 (5,
  '0.053*"need" + 0.028*"show" + 0.028*"bit" + 0.027*"artifa

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.017843090532

Coherence Score:  0.32128687228891173


In [17]:
# Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
# vis

In [ ]:
# won't run or takes too long

# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
import os
os.environ.update({'MALLET_HOME':r'C:/Users/HP/mallet-2.0.8/'})
mallet_path = r'C:\Users\HP\mallet-2.0.8\bin\mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=id2word)

In [19]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('tour', 0.12055791388720437),
   ('trip', 0.09678593086719224),
   ('experience', 0.04536082474226804),
   ('travel', 0.04135839902971498),
   ('tribe', 0.033474833232261976),
   ('book', 0.021710127349909035),
   ('start', 0.02110369921164342),
   ('arrange', 0.019769557307459065),
   ('meet', 0.019648271679805943),
   ('organise', 0.016616130988477865)]),
 (1,
  [('build', 0.04173354735152488),
   ('painting', 0.0342017533028769),
   ('building', 0.024694406716878628),
   ('live', 0.023830102481787876),
   ('carve', 0.023336214347450302),
   ('ground', 0.02247191011235955),
   ('site', 0.02136066181010001),
   ('religious', 0.020990245709346832),
   ('emperor', 0.02086677367576244),
   ('unique', 0.020249413507840475)]),
 (2,
  [('find', 0.06309345327336331),
   ('small', 0.027486256871564217),
   ('tourist', 0.025112443778110945),
   ('area', 0.02448775612193903),
   ('place', 0.02361319340329835),
   ('pay', 0.022488755622188907),
   ('site', 0.021989005497251374),
   ('ph

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(num_topics)
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

start1 = 2 
limit1 = 11
step1 = 1

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=start1, limit=limit1, step=step1)

x = range(start1, limit1, step1)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

2


In [21]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Num Topics = 2  has Coherence Value of 0.6341
Num Topics = 3  has Coherence Value of 0.6866
Num Topics = 4  has Coherence Value of 0.6958
Num Topics = 5  has Coherence Value of 0.7142
Num Topics = 6  has Coherence Value of 0.7263
Num Topics = 7  has Coherence Value of 0.7063
Num Topics = 8  has Coherence Value of 0.726
Num Topics = 9  has Coherence Value of 0.723
Num Topics = 10  has Coherence Value of 0.7304


In [22]:
optimal_model = model_list[8] # Number of Topics = 10
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.197*"day" + 0.072*"trek" + 0.055*"scenery" + 0.046*"history" + '
  '0.042*"spectacular" + 0.035*"night" + 0.025*"hiking" + 0.019*"wildlife" + '
  '0.015*"inside" + 0.014*"structure"'),
 (1,
  '0.296*"experience" + 0.052*"unforgettable" + 0.044*"make" + '
  '0.043*"painting" + 0.035*"life" + 0.022*"absolutely" + 0.022*"close" + '
  '0.021*"expect" + 0.018*"spot" + 0.012*"country"'),
 (2,
  '0.109*"small" + 0.061*"important" + 0.051*"tribe" + 0.043*"miss" + '
  '0.039*"culture" + 0.032*"expensive" + 0.029*"hour" + 0.024*"unbelievable" + '
  '0.021*"simply" + 0.020*"information"'),
 (3,
  '0.095*"view" + 0.094*"stunning" + 0.041*"learn" + 0.034*"walk" + '
  '0.029*"exhibit" + 0.028*"art" + 0.027*"move" + 0.025*"landscape" + '
  '0.024*"spend" + 0.022*"live"'),
 (4,
  '0.255*"history" + 0.045*"people" + 0.044*"lot" + 0.030*"religious" + '
  '0.028*"part" + 0.023*"display" + 0.018*"cultural" + 0.013*"recent" + '
  '0.013*"stone" + 0.012*"meet"'),
 (5,
  '0.242*"tour" + 0.161*"guid

In [23]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3.0,0.1296,"view, stunning, learn, walk, exhibit, art, mov...",Incredible locations dont be put off thinking ...
1,1,8.0,0.1282,"site, world, historical, unique, historic, tow...",It is a real miracle of the world!
2,2,1.0,0.1154,"experience, unforgettable, make, painting, lif...",Incredible experience
3,3,0.0,0.1000,"day, trek, scenery, history, spectacular, nigh...",Amazing
4,4,4.0,0.1176,"history, people, lot, religious, part, display...",So much history
5,5,2.0,0.1068,"small, important, tribe, miss, culture, expens...","A definite ""GO!"""
6,6,0.0,0.1000,"day, trek, scenery, history, spectacular, nigh...",Just WoW and Amazing !!
7,7,0.0,0.1000,"day, trek, scenery, history, spectacular, nigh...",Amazing
8,8,7.0,0.1133,"trip, highlight, architecture, building, area,...",Amnazing
9,9,0.0,0.1000,"day, trek, scenery, history, spectacular, nigh...",wonderful place


In [24]:
df_dominant_topic['Dominant_Topic'].value_counts()

0.0    3819
1.0     618
2.0     543
3.0     523
4.0     513
6.0     508
5.0     495
8.0     443
7.0     439
9.0     410
Name: Dominant_Topic, dtype: int64

In [25]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.1786,"day, trek, scenery, history, spectacular, nigh...",Great experience on our 2 nights & 3 days trek...
1,1.0,0.1735,"experience, unforgettable, make, painting, lif...",incredible off the beaten track experience we ...
2,2.0,0.1509,"small, important, tribe, miss, culture, expens...",Great national park and unbelievable tribes an...
3,3.0,0.1636,"view, stunning, learn, walk, exhibit, art, mov...","Great exhibit on Lucy, human evolution and ani..."
4,4.0,0.1528,"history, people, lot, religious, part, display...",An engaging lesson and walk through Ethiopias ...
5,5.0,0.1724,"tour, guide, local, move, city, spiritual, rec...",Avoid! avoid! Avoid! Abusive and DANGEROUS Gui...
6,6.0,0.1461,"worth, time, visit, stop, cave, adventure, qui...",Worth a stop en route to Tomoca
7,7.0,0.1607,"trip, highlight, architecture, building, area,...","Big city , main tourist center because of the ..."
8,8.0,0.1667,"site, world, historical, unique, historic, tow...","Unique site, poorly managed"
9,9.0,0.1509,"incredible, informative, top, human, ancient, ...",The below ground tombs would have been great i...


In [26]:
sent_topics_sorteddf_mallet['Text'].values

array(['Great experience on our 2 nights & 3 days trekking w/ spectacular scenery',
       'incredible off the beaten track experience we had great guide who made sure our experience was unforgettable',
       'Great national park and unbelievable tribes and cultures ',
       'Great exhibit on Lucy, human evolution and animal evolution!',
       'An engaging lesson and walk through Ethiopias recent history',
       'Avoid! avoid! Avoid! Abusive and DANGEROUS Guide! Reported to the Authorities in Ethiopia -',
       'Worth a stop en route to Tomoca',
       'Big city , main tourist center because of the area ',
       'Unique site, poorly managed',
       'The below ground tombs would have been great if I had a flashlight.'],
      dtype=object)

In [27]:
# # Number of Documents for Each Topic
# topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# # Percentage of Documents for Each Topic
# topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# # Topic Number and Keywords
# topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# # Concatenate Column wise
# df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# # Change Column names
# df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# # Show
# df_dominant_topics